# Options IV Calculation Test

This notebook tests the options data fetching and implied volatility (IV) calculation pipeline.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from adapters.options_adapter import OptionsAdapter
from adapters.rates_adapter import RatesAdapter
from adapters.ticker_adapter import TickerAdapter
from models.options_data import OptionsRequest, OptionType
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from engines.IV_smile import IVEngine


## Setup

Initialize the adapter and create the options request.


In [3]:
adapter = OptionsAdapter()

# Simulate duration selection (e.g. '1y' from frontend)
today = date.today()
expiry_end = today + relativedelta(weeks=16)

req = OptionsRequest(
    ticker="AAPL",
    optionType=OptionType.CALL,
    expiryStart=today,
    expiryEnd=expiry_end,
    strikeMin=250,
    strikeMax=310,
)

## Fetch Options Data


In [4]:
print(f"Requesting options for {req.ticker} until {expiry_end}...")
df = adapter.fetch_option_chain(req)
print(f"Fetched {len(df)} contracts.")



Requesting options for AAPL until 2026-04-06...
Fetched 148 contracts.


## Display Options Data


In [5]:
df[["optionType", "strike", "timeToExpiry", "midPrice", "expiry"]]

,optionType,strike,timeToExpiry,midPrice,expiry
0,call,250.0,0.010785,24.855,2025-12-19
1,call,252.5,0.010785,21.955,2025-12-19
2,call,255.0,0.010785,19.520,2025-12-19
3,call,257.5,0.010785,16.945,2025-12-19
4,call,260.0,0.010785,14.600,2025-12-19
...,...,...,...,...,...
143,call,290.0,0.259986,7.605,2026-03-20
144,call,295.0,0.259986,6.030,2026-03-20
145,call,300.0,0.259986,4.640,2026-03-20
146,call,305.0,0.259986,3.590,2026-03-20


## Calculate Implied Volatility


In [15]:
from engines.zero_rates import ZeroRatesEngine
import yfinance as yf

df["rate"] = ZeroRatesEngine.interpolate_zero_rate(df, tte_col="timeToExpiry")

base_info = TickerAdapter.fetchBasic(req.ticker)
print(f"Fetched base info for {req.ticker}.")
div = base_info.dividendYield
print(f"Dividend yield: {div:.4f}%")
print(f"Spot price: ${base_info.spot:.2f}")


Fetched base info for AAPL.
Dividend yield: 0.3700%
Spot price: $274.11


In [16]:
surface_data = IVEngine.generateIVSmile(df, df["rate"], div, spot, OptionType.CALL) #type: ignore
print("\nIV Calculation Results:")
surface_data


IV Calculation Results:


,type,K,T,Price,rate,expiry,iv,S
0,call,250.0,0.010785,24.855,0.037641,2025-12-19,0.822649,274.109985
1,call,252.5,0.010785,21.955,0.037641,2025-12-19,0.695609,274.109985
2,call,255.0,0.010785,19.520,0.037641,2025-12-19,0.644396,274.109985
3,call,257.5,0.010785,16.945,0.037641,2025-12-19,0.570554,274.109985
4,call,260.0,0.010785,14.600,0.037641,2025-12-19,0.528715,274.109985
...,...,...,...,...,...,...,...,...
143,call,290.0,0.259986,7.605,0.036391,2026-03-20,0.396565,274.109985
144,call,295.0,0.259986,6.030,0.036391,2026-03-20,0.381408,274.109985
145,call,300.0,0.259986,4.640,0.036391,2026-03-20,0.365746,274.109985
146,call,305.0,0.259986,3.590,0.036391,2026-03-20,0.354349,274.109985


In [17]:
import numpy as np
surface_data["k"] = np.log(surface_data["K"] / spot)
surface_data["w"] = surface_data["iv"] ** 2 * surface_data["T"]
surface_data[["w", "k"]]

,w,k
0,0.007299,-0.092069
1,0.005219,-0.082118
2,0.004479,-0.072266
3,0.003511,-0.062510
4,0.003015,-0.052848
...,...,...
143,0.040886,0.056351
144,0.037821,0.073446
145,0.034778,0.090253
146,0.032645,0.106782
